In [3]:
import pandas as pd
import plotly.express as px
import numpy as np
import os

segment = 'method' # 'contract' # 'project'

In [5]:
prepend = 'csv_exports/whale_actions/'

op_df = pd.read_csv(prepend + 'op_whale_actions_22_01_18.csv')
op_df['chain'] = 'Optimism'

l1_df = pd.read_csv(prepend + 'l1_whale_actions_22_01_18.csv')
l1_df['chain'] = 'Ethereum'

arb_df = pd.read_csv(prepend + 'arbi_whale_actions_22_01_18.csv')
arb_df['chain'] = 'Arbitrum'

df = pd.concat([op_df, l1_df, arb_df])
df.sample(20)

,address,token_direction,project,contract,method,usd_value,num_tfers,num_txs,chain
162356,0xac9aa340218ef9440fcc42d5e12e75e808ad063f,out,0xc4b6d3a57954fc8021d513e8187e458ec2a20fb0,NaN,transfer,1.511630e+04,2,2,Ethereum
11432,0x136cc089c5d4eadde8159840107d7c67a2215183,out,OTHER,OTHER,OTHER,3.539683e+01,1,1,Ethereum
112257,0x134c8aed8ea0430fc634cc7f2ab2ee0bb9762416,out,0xfb4b165b3baad687f0baeee41313920f85fbf947,NaN,transfer,1.007082e+04,1,1,Ethereum
245225,0x9ac5dde174120e859de5fdb87048b9ce2d936033,out,0x833f3b6faa717079fb3a1030f6207c57b1c591bd,NaN,transfer,2.215580e+06,2,2,Ethereum
108053,0x68a9857b0e540e5a800151d3e4a33037191a5dc4,in,aura_finance,BaseRewardPool,withdraw,3.385741e+05,1,1,Ethereum
129716,0xf60c2ea62edbfe808163751dd0d8693dcb30019c,out,0x517bfb434ceaeae71de7bc22846419a0ce978e3c,NaN,transfer,1.271912e+04,2,2,Ethereum
103810,0x0c7999141ed375808aaf91c52e83638c8c1b66ad,in,muffin_migrator,MuffinHubPositions,multicall,6.237917e+04,2,2,Ethereum
29076,0x91a0e0ceb54f21117707b174fd03faa50e9801bf,out,0xc88f1e608d4cbaa17539c83a2b90f3d399eb0fc1,NaN,transfer,2.339866e+06,15,15,Ethereum
159050,0xbf4a768f3e3a1edd832e43758dce6d671ef76c2c,out,0xcd531ae9efcce479654c4926dec5f6209531ca7b,NaN,transfer,1.506516e+06,1,1,Ethereum
144846,0x26291431c7fb7ea43a7caba9a9e8fcbabd186be8,in,0x460c8ce74c65fa937a46ebf44ab0eaf2e894e469,NaN,transfer,1.007082e+05,1,1,Ethereum


In [6]:

df['usd_value_in'] = np.where(df['token_direction'] == 'in',df['usd_value'], 0)
df['usd_value_out'] = np.where(df['token_direction'] == 'out',df['usd_value'], 0)
df['usd_value_net'] = df['usd_value_in'] - df['usd_value_out']
df['usd_value_net'] = np.where(df['usd_value_net'] < 0, 0, df['usd_value_net'])
val_cols = ['usd_value','usd_value_in','usd_value_out','usd_value_net']

In [7]:
df_addr = df.groupby(['chain','address','project','contract','method']).sum()
df_addr.reset_index(inplace=True)

for val in val_cols:
        df_addr[val + '_pct_share'] = df_addr.groupby('address')[val].transform(lambda x: x / x.sum())
# display(df_addr)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1126789040.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_addr = df.groupby(['chain','address','project','contract','method']).sum()


In [8]:
group_list = ['chain','project','contract','method']
dlist = ['chain','project','contract','method','action',\
                'usd_value_pct_share','usd_value_in_pct_share','usd_value_out_pct_share','usd_value_net_pct_share','address']

if segment == 'project':
        group_list.drop('method',inplace=True)
        group_list.drop('contract',inplace=True)
        dlist.drop('method',inplace=True)
        dlist.drop('contract',inplace=True)
elif segment == 'contract':
        group_list.drop('method',inplace=True)
        dlist.drop('method',inplace=True)

In [9]:
df_dollar = df[['chain','project','contract','method','usd_value','usd_value_in','usd_value_out','usd_value_net','address']]
df_dollar = df_dollar.groupby(['chain','project','contract','method']).agg({
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
        ,'address':'nunique'
})
df_dollar.reset_index(inplace=True)
df_dollar.name = 'df_dollar'
for val in val_cols:
        df_dollar[val + '_pct_share'] =  df_dollar[val] / ( df_dollar[val].sum() )


In [10]:
# Get the average volume share across all whales
# i.e. once someone is determined to be a whale, tkae their pct transfer share, and then we avg across rather than $ weight.
# The reason to not dollar-weight is that we care more about genetating the average individual user experience, rather than the whaliest whale.

df_action = df_addr.groupby(['chain','project','contract','method']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        ,'address':'nunique'
})
df_action.name = 'df_action'

# display(df_action)

In [12]:
dfs = [df_action, df_dollar]


if not os.path.exists("csv_outputs"):
        os.mkdir("csv_outputs")
        
for i, d in enumerate(dfs):
        original_name = d.name
        d.reset_index(inplace=True)
        if segment == 'method':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str) + ' | ' + d['method'].astype(str)
        elif segment == 'contract':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str)
        elif segment == 'project':
                d['action'] = d['project'].astype(str)
        
        d = d[dlist]
        
        dfs[i] = d
        d.name = original_name
        
#dumb hardcode, tried to do it fancy, but whatever
df_action = dfs[0]
df_dollar = dfs[1]

df_action.to_csv('csv_outputs/whale_actions_share.csv')
df_dollar.to_csv('csv_outputs/whale_actions_share_dollar_weight.csv')

df_dollar.sample(20)
# gix = px.pie(df_action, values='usd_value_out_pct_share', names='action', title='Share of Actions')

,chain,project,contract,method,action,usd_value_pct_share,usd_value_in_pct_share,usd_value_out_pct_share,usd_value_net_pct_share,address
1490,Ethereum,liquity,ActivePool,closeTrove,liquity - ActivePool | closeTrove,2.690742e-05,3.099023e-05,0.000000,3.099023e-05,10
1496,Ethereum,liquity,LQTYStaking,stake,liquity - LQTYStaking | stake,7.161568e-06,9.472937e-08,0.000054,9.472937e-08,7
260,Arbitrum,sushi,Pair,swapTokensForExactTokens,sushi - Pair | swapTokensForExactTokens,1.046409e-05,1.122257e-05,0.000005,1.122257e-05,4
1367,Ethereum,goldfinch,MigratedTranchedPool,withdrawMultiple,goldfinch - MigratedTranchedPool | withdrawMul...,1.963172e-07,2.261055e-07,0.000000,2.261055e-07,1
312,Arbitrum,uniswap_v3,Pair,0x009c4001,uniswap_v3 - Pair | 0x009c4001,5.876204e-06,6.767833e-06,0.000000,6.767833e-06,2
713,Ethereum,aave_v2,WrappedTokenGatewayV2,withdrawETH,aave_v2 - WrappedTokenGatewayV2 | withdrawETH,1.474385e-03,1.698102e-03,0.000000,1.698102e-03,99
1895,Ethereum,sushi,Pair,swapOnUniswapV2Fork,sushi - Pair | swapOnUniswapV2Fork,4.025994e-07,1.520799e-07,0.000002,1.520799e-07,4
2374,Optimism,Uniswap,Pool,0x0222e005,Uniswap - Pool | 0x0222e005,7.278113e-06,8.382462e-06,0.000000,8.382462e-06,1
1711,Ethereum,polygon,DepositManager,processExits,polygon - DepositManager | processExits,4.664153e-05,5.371871e-05,0.000000,5.371871e-05,14
255,Arbitrum,sushi,Pair,swapETHForExactTokens,sushi - Pair | swapETHForExactTokens,1.100209e-05,1.267150e-05,0.000000,1.267150e-05,1


In [15]:
chains = df_dollar['chain'].drop_duplicates().to_list()
print('Top Dollar Weighted Actions by Chain')

for c in chains:
        tdf = df_dollar[df_dollar['chain'] == c]
        sumdf = tdf.groupby(['chain','project','action']).sum()
        sumdf.reset_index(inplace=True)
        sumdf.sort_values(by = '')

['Arbitrum', 'Ethereum', 'Optimism']
Top Dollar Weighted Actions by Chain
